### Final Project for Applied Computation
Compiled by Cole Speed <br>
Fall 2021<br>
University of Texas at Austin

### Motivation and Goal
Numerous dune fields are visible across the surface of Mars. While the distribution of dune fields is relatively well-known, the form and composition of Martian dune fields remains poorly understood. Here we seek to develop a workflow that integrates very high-resolution thermal imagery of Mars and hand-mapped dune field locations to quantify relationships between dune field type and mean daytime temperature for dune fields across the Red Planet.

### Datasets
Datasets being used: <br>
(1) Thermal Emmission Imaging System (THEMIS) global mosaic (60S - 60N ) at 100 m/pixel resolution<br>
(2) Mars Global Digital Dune Database (MGD3) vector shapefile for dune field locations <br>

### Workflow/Approach
1. Get the data in the same spatial reference (Project)
2. Add numeric field to shapefile attribute table for classification by “dune type” (e.g. Barchan, Transverse, etc.)
3. Create global raster of dune fields by ‘rasterizing’ dune field polygons based on “dune type”
4. Calculate Zonal Statistics for dune field areas temperature stats for each “dune_type”. 
5. Investigate spatial relationships using Spatial Autocorrelation.

### Import Packages and Set Environmental Variables

In [1]:
import arcpy
from arcpy import env
import os
import numpy as np
import geopandas as gpd
from arcpy.sa import *

In [2]:
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

In [3]:
path = r'C:\Users\Cole\Dropbox\Courses\Applied_Geocomputation\project\data'

In [4]:
env.workspace = path

In [5]:
arcpy.env.overwriteOutput = True

### Load and inspect shapefiles for dunefields on mars

In [6]:
shplist = [f for f in os.listdir(path+'\shapefiles') if f.endswith("shp")]

In [7]:
shplist

['DF_MGD3.shp']

In [8]:
MGD3 = path+'\shapefiles\\'+shplist[0]

In [9]:
arcpy.Describe(MGD3).spatialReference

type,Geographic
name,GCS_Mars_2000
factoryCode,104905
datumName,D_Mars_2000
angularUnitName,Degree


We see that the spatial references differ. These will be projected to match in the "Data Processing" portion of the notebook.

In [10]:
desc_MGD3 = arcpy.Describe(MGD3)

In [11]:
for field in desc_MGD3.fields:
    print("%-22s %s %s" % (field.name, ":", field.type))
    #print field.name + " = " + field.type

FID                    : OID
Shape                  : Geometry
Shape_Leng             : Double
Shape_Area             : Double
Dune_Lon               : Double
Dune_Lat               : Double
Dune_ID                : String
Dune_Type              : String
B_1                    : String
Bd_1                   : String
D_1                    : String
L_1                    : String
S_1                    : String
SS_1                   : String
T_1                    : String
U_1                    : String
Confidence             : String
Type_Image             : String
Area_sinu_             : Double
MDHeight               : Double
Volume_1               : Double
Volume_2               : Double
Avg_El                 : Integer
CcDcAzimut             : Double
SF_1_Az                : Double
SF_1_Count             : SmallInteger
SF_2_Azimu             : Double
SF_2_Count             : SmallInteger
SF_3_Azimu             : Double
SF_3_Count             : SmallInteger
SF_4_Azimu            

It will be easier to visualize the kind of data we are working by viewing the first few rows of the attribute table. To do this we create a geopandas dataframe for our shapefile. 

In [12]:
gdf = gpd.read_file(MGD3)
gdf.head()

,Shape_Leng,Shape_Area,Dune_Lon,Dune_Lat,Dune_ID,Dune_Type,B_1,Bd_1,D_1,L_1,...,Cr_Lat,Crater_BID,Cr_Area_si,Cr_Diam_1,Mars_5M_Ch,IR,VIS,MOC,Comments_1,geometry
0,149822.358907,4.242661e+08,28.963860,50.116277,0289+501,"B, Bd",1,1,0,0,...,50.359016,0292+503,35698.749656,213.197322,5,I01522009,"V01185004, V01522010, V13492005","E0302501, M1900242",None,"POLYGON ((28.95935 50.01387, 28.94613 50.00412..."
1,18156.580334,1.346222e+07,59.766060,36.991779,0597+369,"Bd, U",0,1,0,0,...,36.981607,0598+369,186.160618,15.395690,5,"I02507005, I04392008",V02507006,E1004763,None,"POLYGON ((59.72737 37.02275, 59.72838 37.02275..."
2,352484.404425,6.778119e+08,44.349411,41.722687,0443+417,"B, Bd",1,1,0,0,...,41.802380,0446+418,12430.821804,125.807103,5,"I01122002, I03656007","V01122003, V10384008, V11919009, V12518004","E0100550, E0300309, M1202264, R0200939",None,"POLYGON ((43.96887 41.76218, 43.97274 41.76887..."
3,23597.611733,1.486876e+07,64.980467,34.240573,0649+342,"B, Bd, T",1,1,0,0,...,0.000000,0,0.000000,0.000000,6,I01995006,V09909013,None,None,"POLYGON ((65.00043 34.27721, 65.00094 34.26580..."
4,67519.564651,1.759504e+08,12.955661,15.997274,0129+159,SS,0,0,0,0,...,16.085122,0129+160,5002.952526,79.812044,12,I08195012,"V01797007, V11271005, V11583005,","M0902670, R1004453",dune truncated at image margin,"POLYGON ((12.97881 16.12480, 12.99243 16.05120..."


The "Dune_Type" field is the field of interest, but it is a string which is not useful for raster classification. The next 8 fields further describe the type of dune field with a 0 if that type of dune isn't present and a 1 if it is. These columns are more useful to us, but it we need them as numeric values. We will create a new field "Kind_of_Dune" which will have 8-bit integer type representing the type of dune field. An arcpy SearchCursor is used to populate the "Kind_of_Dune" field with a value corresponding to the dune type. This will be completed in "Data Processing" below.

### Load Mars THEMIS raster
This takes a while...it is a 100m/pixel raster covering the entire equatorial region of Mars (60S-60N) and is 22GB!!!

In [13]:
themis_day = path + r'\rasters\Mars_MO_THEMIS-IR-Day_mosaic_global_100m_v12.tif'

In [14]:
dtype = arcpy.GetRasterProperties_management(themis_day, "VALUETYPE")
dtvalue = dtype.getOutput(0)
print(f"Data type is: {dtvalue}")

Data type is: 3


In [15]:
dtype_dict = {0: "1-bit",
              1: "2-bit",
              2: "4-bit",
              3: "8-bit integer"}

In [16]:
print(f"Data type is: {dtype_dict.get(int(dtvalue))}")

Data type is: 8-bit integer


In [17]:
cell_prop = arcpy.GetRasterProperties_management(themis_day, "CELLSIZEX")
csize = cell_prop.getOutput(0)
print(f"Cell size: {csize} meters")

Cell size: 100 meters


In [18]:
themis_spatial_ref = arcpy.Describe(themis_day).spatialReference
themis_spatial_ref

type,Projected
name,SimpleCylindrical_Mars
factoryCode,N/A
linearUnitName,Meter
GCS.name,GCS_Mars


In [19]:
simpCyl = themis_spatial_ref

In [20]:
desc = arcpy.Describe(themis_day)

xmin = desc.extent.XMin
xmax = desc.extent.XMax
ymin = desc.extent.YMin
ymax = desc.extent.YMax

print ("xmin: %s \nxmax: %s \nymin: %s \nymax: %s" % (xmin, xmax, ymin, ymax))

xmin: -10669500.0 
xmax: 10669500.0 
ymin: -5334800.0 
ymax: 5334800.0


We need to ensure that our analysis takes place over the same spatial extent. We can do so by using the extent of the .png raster we are using.

### Set environmental extent

In [21]:
# Set the extent environment using a keyword.
arcpy.env.extent = "MAXOF"

# Set the extent environment using the Extent class.
arcpy.env.extent = arcpy.Extent(xmin, ymin, xmax, ymax)

The THEMIS image is a single band 8-bit integer with values ranging from 0-255 corresponding to intensity values representing average daytime temperatures on the martian surface.

### Data processing

The CRS for the files do not match, so we need to project the shapefile data to match the raster data.

In [23]:
outf = MGD3[0:84] + '_simpleCylindrical.shp'
arcpy.management.Project(MGD3, outf, simpCyl)

<Result 'C:\\Users\\Cole\\Dropbox\\Courses\\Applied_Geocomputation\\project\\data\\shapefiles\\DF_MGD3_simpleCylindrical.shp'>

In [24]:
MGD3_simpCyl = path+'\shapefiles\\DF_MGD3_simpleCylindrical.shp'

First we add a field which will store the kind of dunefield for each feature. There are 8 kinds, so a short integer will suffice.

In [25]:
arcpy.AddField_management(MGD3_simpCyl, "dune_kind", 'SHORT', 1)

<Result 'C:\\Users\\Cole\\Dropbox\\Courses\\Applied_Geocomputation\\project\\data\\shapefiles\\DF_MGD3_simpleCylindrical.shp'>

Using a Cursor and a combination of **for** and **if** loops we can populate this field by examining each of the dunetype fields and compiling the result into the 'dune_kind' field.

In [26]:
with arcpy.da.UpdateCursor(MGD3_simpCyl, ['B_1', 'Bd_1', 'D_1', 'L_1', 'S_1', 'SS_1', 'T_1', 'U_1','dune_kind']) as cursor:
    
    for r in cursor:
        
        tot = 0
        
        if int(r[0]) == 1:
            
            tot = tot + 1
            dune_type = 0
            
        if int(r[1]) == 1:
            
            tot = tot + 1
            dune_type = 1
        
        if int(r[2]) == 1:
            
            tot = tot + 1 
            dune_type = 2
       
        if int(r[3]) == 1:
            
            tot = tot + 1 
            dune_type = 3
        
        if int(r[4]) == 1:
            
            tot = tot + 1 
            dune_type = 4
        
        if int(r[5]) == 1:
            
            tot = tot + 1
            dune_type = 5
        
        if int(r[6]) == 1:
            
            tot = tot + 1 
            dune_type = 6
            
        if int(r[7]) == 1:
            
            tot = tot + 1 
            dune_type = 7
        
        if tot > 1:
            
            dune_type = 9 #mixed dune field

        else:
            
            dune_type = dune_type
            
        
        r[8] = int(dune_type)
        cursor.updateRow(r)
    

Print summary statistics. Shows overwhelmingly that 'mixed' dune fields are most common. May need to further break them out and look at the results

In [27]:
gdf = gpd.read_file(MGD3_simpCyl)
gdf.head()

,Shape_Leng,Shape_Area,Dune_Lon,Dune_Lat,Dune_ID,Dune_Type,B_1,Bd_1,D_1,L_1,...,Crater_BID,Cr_Area_si,Cr_Diam_1,Mars_5M_Ch,IR,VIS,MOC,Comments_1,dune_kind,geometry
0,149822.358907,4.242661e+08,28.963860,50.116277,0289+501,"B, Bd",1,1,0,0,...,0292+503,35698.749656,213.197322,5,I01522009,"V01185004, V01522010, V13492005","E0302501, M1900242",None,9,"POLYGON ((-8952888.976 2964557.248, -8953672.5..."
1,18156.580334,1.346222e+07,59.766060,36.991779,0597+369,"Bd, U",0,1,0,0,...,0598+369,186.160618,15.395690,5,"I02507005, I04392008",V02507006,E1004763,None,9,"POLYGON ((-7129123.777 2194512.096, -7129064.0..."
2,352484.404425,6.778119e+08,44.349411,41.722687,0443+417,"B, Bd",1,1,0,0,...,0446+418,12430.821804,125.807103,5,"I01122002, I03656007","V01122003, V10384008, V11919009, V12518004","E0100550, E0300309, M1202264, R0200939",None,9,"POLYGON ((-8063204.350 2475440.682, -8062974.8..."
3,23597.611733,1.486876e+07,64.980467,34.240573,0649+342,"B, Bd, T",1,1,0,0,...,0,0.000000,0.000000,6,I01995006,V09909013,None,None,9,"POLYGON ((-6816564.794 2031770.971, -6816534.7..."
4,67519.564651,1.759504e+08,12.955661,15.997274,0129+159,SS,0,0,0,0,...,0129+160,5002.952526,79.812044,12,I08195012,"V01797007, V11271005, V11583005,","M0902670, R1004453",dune truncated at image margin,5,"POLYGON ((-9900130.759 955792.400, -9899322.90..."


Now let's reproject the shapefile data to match the THEMIS raster data

As noted above, there is a field with type string containing the dune field "type", and there are numeric fields with the number 1 denoting the dune field type. However, for this to be useful to us, we need a single field on which to 'rasterize' our vector data by 'dunefield type'. There are several dune fields labeled to contain multiple types of dunes - we will call these simply "mixed dunes". So here we add a field called "dune type" which will have values ranging from 1-4; (1)Barchan dunes; (2)Cresentic dunes; (3); and (4) Mixed dunes 

### Rasterize the Dune Field Polygons

In [36]:
# Set local variables
inFeatures = MGD3_simpCyl; valField = "dune_kind"; outRaster = path+ r'\rasters\MGD3_duneKind.tif';
assignmentType = "CELL_CENTER"; priorityField = ""; cellSize = 100

In [37]:
arcpy.conversion.PolygonToRaster(inFeatures, valField, outRaster, assignmentType, priorityField, cellSize)

<Result 'C:\\Users\\Cole\\Dropbox\\Courses\\Applied_Geocomputation\\project\\data\\rasters\\MGD3_duneKind.tif'>

### Compute Zonal Statitistics for THEMIS pixel values for zone in our Dune Type raster
Each pixel in the THEMIS raster corresponds to a thermal intensity. We are interested in how thermal intensity varies between dune field types. So here we get the thermal intensity values at each pixel location for the dune type raster.

We first must add a field to our dune type raster attribute table. We are also interested in the total coverage of each dune type, so we add a field for area.

In [28]:
dune_type_rast = path + r'\rasters\MGD3_duneKind.tif'

We will use Zonal Statistics to determine the THEMIS intensity values for each given Dune Type class. Per [this resource](https://help.arcgis.com/en/arcgisdesktop/10.0/help/index.html#//009z000000w7000000.htm), computing zonal statistics requires both input rasters to have an attribute table. So we create one for the THEMIS dataset below (Dune Type raster already has an attribute table). 

In [29]:
daytime_stats = arcpy.BuildRasterAttributeTable_management(themis_day, "Overwrite")

In [30]:
Day_ZonalStats = ZonalStatisticsAsTable(dune_type_rast, "Value", themis_day, "daytime_dune_stats.dbf", "", "ALL")

In [31]:
# convert to excel sheet
arcpy.conversion.TableToDBASE(Day_ZonalStats, path)

<Result 'C:\\Users\\Cole\\Dropbox\\Courses\\Applied_Geocomputation\\project\\data'>

### Spatial Autocorrelation
In order to investigate whether dune fields types show any internal spatial distribution patterns, we use the arcpy Spatial Autocorrelation method. This produces values for Moran I Index, Expected Index, P-, and Z-values. 

In [36]:
arcpy.stats.SpatialAutocorrelation(MGD3_simpCyl, 'dune_kind', "GENERATE_REPORT", "INVERSE_DISTANCE","EUCLIDEAN_DISTANCE")

id,value
0,-0.002907
1,-0.078974
2,0.937053
3,C:\Users\Cole\AppData\Local\Temp\MoransI_Result_10740_2080_1.html
